In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Токенизация в NLTK

[NLTK](https://www.nltk.org) - хорошая библиотека, в ней есть много всего - токенизация, лемматизация, обертки под парсеры, метрики, генераторы...

Есть и недостатки:
* многие инструменты есть только для английского
* сомнительного качества документация, иногда приходится лезть в исходный код

Тем не менее, NLTK - точно лучше, чем писать велосипеды на костылях руками.

## Токенизация

* [документация, где очень много всего и описано много токенизаторов](https://www.nltk.org/api/nltk.tokenize.html)
* [исходный код самого общего токенизатора](https://www.nltk.org/_modules/nltk/tokenize.html)
* [примеры на два самых общих токенизатора](http://www.nltk.org/howto/tokenize.html)

Самая общая вещь, которую можно применить если у вас просто текст (в котором не надо парсить, например, #самыедлинныетэгивмире) - функция `word_tokenize` из `nltk.tokenize`. Она написана для английского и обрабатывает много сложных случаев, для русского - надо смотреть, что нужно для задачи, и, возможно, докручивать.

В `word_tokenize` надо подавать строку с предложением. Есть второй параметр - язык, но русского нет, поэтому язык можно не указывать, по умолчанию там стоит английский.

## Деление на предложения
Еще полезная функция, которую вызывает внутри себя `word_tokenize`, но которой можно также пользоваться - `sent_tokenize`. Эта функция делит текст не на слова, а на предложения. Это называется "сплиттер". Пользоваться ей можно так же как `word_tokenize`.

In [6]:
from nltk.tokenize import sent_tokenize, word_tokenize

### Задание 1.
`sents` - список предложений с разными сложными случаями. Токенизируйте каждое предложение и выведите результат в каком-нибудь удобно читаемом виде.

In [24]:
sents = [
    "On a $50,000 mortgage of 30 years at 8 percent, the monthly payment would be $366.88.",
    "\"We beat some pretty good teams to get here,\" Slocum said.",
    "Well, we couldn't have this predictable, cliche-ridden, \"Touched by an Angel\" (a show creator John Masius worked on) wanna-be if she didn't.",
    "I cannot work under these conditions!!!",
    "He arrived at 3:00 pm.",
    "I walked down the Oxford St. and bought cool shoes."
]

# YOUR CODE HERE

`multi_sents` - это две строки, где содержатся по два предложения. Разбейте их на предложения, а предложения на слова.

In [25]:
multi_sents = [
    "Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.",
    "I called Dr. Jones. I called Dr. Jones."
]

# YOUR CODE HERE

Вопросы к обсуждению:
1. обработку каких разных случаев вы видите?
2. какие, может быть, есть проблемы?

### Задание 2.
Попробуйте придумать похожие сложные случаи для русского (и в целом) и посмотрите, справляется ли с ними этот токенизатор.<br>
(Я знаю несколько моментов, где результат его работы не всегда может считаться корректным. Возможно, вы найдете другие сложности.)

In [3]:
# YOUR CODE HERE

## Индексы токенов

Индексы токенов, если токены уже есть, можно получить с помощью функции `align_tokens`, которая принимает на вход список токенов и исходный текст-строку.

In [10]:
from nltk.tokenize.util import align_tokens

s = "On a $50,000 mortgage of 30 years at 8 percent, the monthly payment would be $366.88."
tokens = word_tokenize(s)
spans = align_tokens(tokens, s)
print(spans)

[(0, 2), (3, 4), (5, 6), (6, 12), (13, 21), (22, 24), (25, 27), (28, 33), (34, 36), (37, 38), (39, 46), (46, 47), (48, 51), (52, 59), (60, 67), (68, 73), (74, 76), (77, 78), (78, 84), (84, 85)]


### Задание 3.
Посмотрите, что происходит с предложением с кавычками. Подумайте, как это можно исправить.

In [12]:
s = "\"We beat some pretty good teams to get here,\" Slocum said."
# YOUR CODE HERE

# Лемматизация

Наши предложения на сегодня:

In [28]:
sent = 'ВКС 27 июля обнаружили и уничтожили запущенный с территории боевиков беспилотник, приближавшийся к авиабазе.'
unkn_sent = 'Я пофиксил баг в продакшене.' # предложение с незнакомыми словами

# омонимия
homonym1 = 'За время обучения я прослушал больше сорока курсов.'
homonym2 = 'Сорока своровала блестящее украшение со стола.'

## [mystem](https://tech.yandex.ru/mystem/)
Как запускать:
* можно скачать mystem и вызывать его [из командной строки с разными параметрами](https://tech.yandex.ru/mystem/doc/)
* можно исполнять команды из пункта выше через питон (см. модуль `subprocess`)
* [pymystem3](https://pythonhosted.org/pymystem3/pymystem3.html) - обертка для питона, работает медленнее, но это удобно.

Сегодня мы будем работать с pymystem3.


In [29]:
from pymystem3 import Mystem
mystem_analyzer = Mystem()

Сейчас мы запустили Mystem c дефолтными параметрами. А вообще параметры есть такие:
* mystem_bin - путь к `mystem`, можно не указывать.
* grammar_info - нужна ли грамматическая информация или только леммы (по дефолту нужна)
* disambiguation - нужно ли снятие омонимии - дизамбигуация (по дефолту нужна)
* entire_input - нужно ли сохранять в выводе все (пробелы всякие, например), или можно выкинуть (по дефолту оставляется все)

Несколько моментов:
* В Mystem нужно подавать строку, токенизатор вшит внутри. Можно, конечно, и пословно анализировать, но тогда он не сможет учитывать контекст.
* По возможности Mystem (и любые другие вещи этого рода) нужно инициализировать один раз, потому что инициализация занимает время и память.

Можно просто лемматизировать текст.

In [30]:
print(mystem_analyzer.lemmatize(sent))

['ВКС', ' ', '27', ' ', 'июль', ' ', 'обнаруживать', ' ', 'и', ' ', 'уничтожать', ' ', 'запущенный', ' ', 'с', ' ', 'территория', ' ', 'боевик', ' ', 'беспилотник', ', ', 'приближаться', ' ', 'к', ' ', 'авиабаза', '.', '\n']


А можно получить грамматическую информацию.

In [31]:
mystem_analyzer.analyze(sent)

[{'analysis': [], 'text': 'ВКС'},
 {'text': ' '},
 {'text': '27'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,неод=род,ед', 'lex': 'июль', 'wt': 0.997934103}],
  'text': 'июля'},
 {'text': ' '},
 {'analysis': [{'gr': 'V,пе=прош,мн,изъяв,сов',
    'lex': 'обнаруживать',
    'wt': 1}],
  'text': 'обнаружили'},
 {'text': ' '},
 {'analysis': [{'gr': 'CONJ=', 'lex': 'и', 'wt': 0.9999770522}], 'text': 'и'},
 {'text': ' '},
 {'analysis': [{'gr': 'V,пе=прош,мн,изъяв,сов', 'lex': 'уничтожать', 'wt': 1}],
  'text': 'уничтожили'},
 {'text': ' '},
 {'analysis': [{'gr': 'A=(вин,ед,полн,муж,неод|им,ед,полн,муж)',
    'lex': 'запущенный',
    'wt': 0.9916251898}],
  'text': 'запущенный'},
 {'text': ' '},
 {'analysis': [{'gr': 'PR=', 'lex': 'с', 'wt': 0.9999778271}], 'text': 'с'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,жен,неод=(пр,ед|вин,мн|дат,ед|род,ед|им,мн)',
    'lex': 'территория',
    'wt': 1}],
  'text': 'территории'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,од=(вин,мн|род,мн)',
    

### Задание.

Попробуйте инициализировать Mystem с разными параметрами и посмотреть, что выходит с морфологическим анализом предложения. Конкретные вопросы:
1. Что происходит с дизамбигуацией?
2. Как хранится грамматическая информация?


In [ ]:
# YOUR CODE HERE

### Другие фичи
Незнакомые слова:

In [32]:
mystem_analyzer.analyze(unkn_sent)

[{'analysis': [{'gr': 'SPRO,ед,1-л=им', 'lex': 'я', 'wt': 0.9999716282}],
  'text': 'Я'},
 {'text': ' '},
 {'analysis': [{'gr': 'V,несов,пе=прош,ед,изъяв,муж',
    'lex': 'пофиксить',
    'qual': 'bastard',
    'wt': 0.6996285915}],
  'text': 'пофиксил'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,гео,муж,неод=(вин,ед|им,ед)',
    'lex': 'баг',
    'qual': 'bastard',
    'wt': 1}],
  'text': 'баг'},
 {'text': ' '},
 {'analysis': [{'gr': 'PR=', 'lex': 'в', 'wt': 0.9999917746}], 'text': 'в'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,неод=пр,ед',
    'lex': 'продакшень',
    'qual': 'bastard',
    'wt': 0.2241225541}],
  'text': 'продакшене'},
 {'text': '.'},
 {'text': '\n'}]

## [pymorphy2](http://pymorphy2.readthedocs.io/en/latest/)
Это модуль на питоне (то есть всё организовано через ООП), довольно быстрый и с кучей функций.

Как запускать:

In [15]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

pymorphy2 работает с отдельными словами. Если дать ему на вход предложение - он его просто не лемматизирует, тк не понимает.

In [34]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(sent)
tokens

['ВКС',
 '27',
 'июля',
 'обнаружили',
 'и',
 'уничтожили',
 'запущенный',
 'с',
 'территории',
 'боевиков',
 'беспилотник',
 ',',
 'приближавшийся',
 'к',
 'авиабазе',
 '.']

In [35]:
analysis = pymorphy2_analyzer.parse(tokens[8])
analysis

[Parse(word='территории', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='территория', score=0.663522, methods_stack=((DictionaryAnalyzer(), 'территории', 41, 6),)),
 Parse(word='территории', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='территория', score=0.216981, methods_stack=((DictionaryAnalyzer(), 'территории', 41, 1),)),
 Parse(word='территории', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='территория', score=0.047169, methods_stack=((DictionaryAnalyzer(), 'территории', 41, 10),)),
 Parse(word='территории', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='территория', score=0.037735, methods_stack=((DictionaryAnalyzer(), 'территории', 41, 7),)),
 Parse(word='территории', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='территория', score=0.034591, methods_stack=((DictionaryAnalyzer(), 'территории', 41, 2),))]

In [36]:
print(analysis[0].normal_form) # лемма
print(analysis[0].tag) # тэг
print(analysis[0].tag.POS) # часть речи

территория
NOUN,inan,femn sing,loct
NOUN


### Задание.
Напишите лемматизацию предложения `sent` через pymorphy2. На выходе должен быть массив лемм.
Сравните лемматизацию с предложенной mystem.

In [38]:
# YOUR CODE HERE

## Другие фичи

Незнакомые слова:

In [39]:
lemmata = []
for token in word_tokenize(unkn_sent):
    ana = pymorphy2_analyzer.parse(token)[0]
    lemmata.append((ana.normal_form, ana.tag, ana.methods_stack))
lemmata

[('я',
  OpencorporaTag('NPRO,1per sing,nomn'),
  ((DictionaryAnalyzer(), 'я', 3152, 0),)),
 ('пофиксила',
  OpencorporaTag('NOUN,inan,femn plur,gent'),
  ((DictionaryAnalyzer(), 'сил', 55, 8),
   (UnknownPrefixAnalyzer(score_multiplier=0.5), 'пофик'))),
 ('баг',
  OpencorporaTag('NOUN,inan,masc sing,accs'),
  ((DictionaryAnalyzer(), 'баг', 19, 3),)),
 ('в', OpencorporaTag('PREP'), ((DictionaryAnalyzer(), 'в', 378, 0),)),
 ('продакшен',
  OpencorporaTag('NOUN,inan,masc,Geox sing,loct'),
  ((FakeDictionary(), 'продакшене', 33, 5),
   (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'шене'))),
 ('.', OpencorporaTag('PNCT'), ((PunctuationAnalyzer(score=0.9), '.'),))]

Склонение слов и согласование слов с числительными:

In [42]:
loc_analysis = analysis[0]
print('inflection:', loc_analysis.inflect({'accs'}).word)
print('locative + 1:', loc_analysis.make_agree_with_number(1).word)
print('locative + 3:', loc_analysis.make_agree_with_number(3).word)
print('locative + 5:', loc_analysis.make_agree_with_number(5).word)

nom_analysis = loc_analysis.inflect({'nomn'})
print('nominative + 1:',nom_analysis.make_agree_with_number(1).word)
print('nominative + 3:',nom_analysis.make_agree_with_number(3).word)
print('nominative + 5:',nom_analysis.make_agree_with_number(5).word)

inflection: территорию
locative + 1: территории
locative + 3: территориях
locative + 5: территориях
nominative + 1: территория
nominative + 3: территории
nominative + 5: территорий


### Снятие омонимии
mystem умеет снимать омонимию по контексту (хотя не всегда преуспевает), pymorphy2 берет на вход одно слово и соответственно вообще не умеет дизамбигуировать по контексту.

In [43]:
mystem_analyzer = Mystem() # инициализирую объект снова, потому что было задание на разные параметры, я хочу дефолтные

print(mystem_analyzer.analyze(homonym1)[-5])
print(mystem_analyzer.analyze(homonym2)[0])

{'text': 'сорока', 'analysis': [{'lex': 'сорок', 'gr': 'NUM=(пр|дат|род|твор)', 'wt': 0.8710292578}]}
{'text': 'Сорока', 'analysis': [{'lex': 'сорока', 'gr': 'S,жен,од=им,ед', 'wt': 0.1210970059}]}


In [44]:
ana1 = pymorphy2_analyzer.parse(homonym1.split(' ')[-2])
ana2 = pymorphy2_analyzer.parse(homonym2.split(' ')[0])
print(ana1 == ana2)
print(ana1[0])

True
Parse(word='сорока', tag=OpencorporaTag('NUMR gent'), normal_form='сорок', score=0.636363, methods_stack=((DictionaryAnalyzer(), 'сорока', 2856, 1),))


# Стоп-слова
Списки стоп-слов для русского есть разные, можно погуглить, а можно взять из nltk. Может быть, вы посчитаете нужным что-то в него добавить.

In [45]:
from nltk.corpus import stopwords
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

# Знаки препинания
Списки знаков препинания тоже уже есть в питоне. Но этот список тоже может понадобиться пополнить.

In [51]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

# Домашнее задание

## Задание 1.
Напишите функцию, предобрабатывающую текст. Она вам пригодится для проекта. В предобработку входит:
* токенизация
* лемматизация
* lowercase
* удаление знаков препинания и стоп-слов

In [ ]:
def preprocessing(text):
    """
    Processes input text for further use.
    :param text: str: input text.
    :return: list[str]: lemmata list
    """
    # YOUR CODE HERE
    pass

## *Задание 2.
Реализуйте подсчет расстояния Левенштейна. Цена замены - 1.

1. Выведите число - расстояние.
  * CONNECT -> CONEHEAD: 4
  * ПРИМЕРЫ -> ПРЕДМЕТ: 4
2. Выведите все необходимые операции, чтобы превратить s1 в s2.  D (англ. delete) — удалить, I (англ. insert) — вставить, R (replace) — заменить, M (match) — совпадение.
  * CONNECT -> CONEHEAD: MMMRIMRR
  * ПРИМЕРЫ -> ПРЕДМЕТ: MMIRMMRD

In [ ]:
def levenshtein_distance(s1, s2):
    """
    Computes Levenshtein distance between s1 and s2.
    :return: int: Levenshtein distance. 
    """
    # YOUR CODE HERE
    pass


def levenshtein_distance_description(s1, s2):
    """
    Computes Levenshtein edit path between s1 and s2.
    :return: str: edit path.
    """
    # YOUR CODE HERE
    pass

## *Задание 3.
Преобразуйте текст в таблицу, где для каждого слова содержится информация по нему: токен, лемма, часть речи (для знаков препинания - PNCT), индексы начала и конца токена в предложении. Например:

```
Коля сдал экзамен, потому что хорошо подготовился.

Коля	коля	NOUN	0	4
сдал	сдать	VERB	5	9
экзамен	экзамен	NOUN	10	17
,	,	PNCT	17	18
потому	потому	ADVB	19	25
что	что	CONJ	26	29
хорошо	хорошо	ADVB	30	36
подготовился	подготовиться	VERB	37	49
.	.	PNCT	49	50
```

В функцию должно быть можно передать аргумент, какой использовать анализатор - `pymorphy2` или `mystem`. Запишите таблицу в csv-файл. csv (comma-separated values) - это формат, в котором в каждой строке записана строка таблицы, а ячейки в строке разделены каким-либо разделителем (запятая, точка с запятой, табуляция (тогда это называется tsv)).

In [25]:
def grammar_table(text, analyzer='pymorphy2'):
    """
    Writes csv with grammar information for given text.
    :param text: str: input text.
    :param analyzer: str: analyzer to use: 'pymorphy2' or 'mystem' 
    """
    # YOUR CODE HERE